In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('apex_model_trt.pth'))

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=0.5,
    steering_offset=0.0,
    throttle_gain=-0.5,
    throttle_offset=0.0
)

In [3]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=1.35, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
log_button = ipywidgets.ToggleButton(value=False, description='enable logging')

display(
    steering_gain,
    steering_offset,
    log_button
)

FloatSlider(value=1.35, description='steering gain', max=3.0, step=0.001)

FloatSlider(value=0.0, description='steering offset', max=0.5, min=-0.5, step=0.001)

ToggleButton(value=False, description='enable logging')

In [4]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=60)

camera.running = True

In [11]:
car.throttle = 0.13

In [13]:
import cv2
import os
from utils import preprocess

LOG_DIRECTORY = 'circuit_log'

if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)
    
count = 0

def execute(change):
    global count, LOG_DIRECTORY
    image = change['new']
    data = preprocess(image)
    xy = model_trt(data)[0].cpu()
    x = float(xy[0])
    car.steering = x * steering_gain.value + steering_offset.value
    
    if log_button.value:
        image_path = os.path.join(LOG_DIRECTORY, '%012d_%03d.jpg' % (count, round(50 * x + 50)))
        cv2.imwrite(image_path, image)
        count += 1
    else:
        count = 0

In [14]:
execute({'new': camera.value})

In [15]:
camera.observe(execute, names='value')

In [12]:
camera.unobserve_all()